In [1]:
import functools as fn
import itertools as itr
from importlib import reload

import numba as nb
import numpy as np
import numpy.linalg as la
import pandas as pd
import pyitlib.discrete_random_variable as drv
import scipy.spatial.distance as dist
import scipy.stats as st

import entropy as ent
import parametersearch as ps
import score as sc
import utility as utl

<IPython.core.display.Javascript object>

In [2]:
# load user profile data
df = pd.read_feather('../processed_data/profiles.feather')
row_missing_count = df['missing_count']
df = df.drop(columns='missing_count')

# recode missing values as np.nan
MISSING_IND = np.nan
df = df.replace(to_replace=-1, value=MISSING_IND)

missing_mask = fn.partial(ent._missing_mask, missing_ind=MISSING_IND)
pdist = fn.partial(utl.pdist, n_jobs=-1)

<IPython.core.display.Javascript object>

In [3]:
# check for missing data
missing_rate = df.isna().mean()
missing_rate.describe()

count    2603.000000
mean        0.760982
std         0.192352
min         0.000000
25%         0.666145
50%         0.807740
75%         0.905633
max         0.998523
dtype: float64

<IPython.core.display.Javascript object>

In [4]:
# check for columns with less than half missing values
at_least_half = missing_rate[missing_rate < 0.5]
n_less_than_half = len(missing_rate) - len(at_least_half)
p_less_than_half = n_less_than_half / len(missing_rate)
print(
    f'{n_less_than_half} columns ({p_less_than_half:.0%}) are empty in at least half the rows.'
)

total_missing = missing_rate.mean()
print(f'{total_missing:.0%} of all cells are empty.')

2382 columns (92%) are empty in at least half the rows.
76% of all cells are empty.


<IPython.core.display.Javascript object>

This dataset is dominated by missing values: 77% of all cells are empty. 93% of variables are empty for at least half the profiles. Most users don't answer most questions. The values are missing not-at-random (MNAR):

- Users each answer different numbers of questions.
- OKCupid prioritizes some questions over others in the order they are presented to users.
- Users can choose to not answer any particular question, or to hide their response.

Treating missing values in each variable as a distinct informative class would likely grossly distort similarities between users. Imputation would be problematic with data MNAR and with the volume of missing data. It will be more appropriate to use methods that can tolerate missing values as an absence of information.

The existing popular functions for calculating entropy on data do not handle missing values well. Simply calculating entropy on the remaining data for a variable as if there were no missing values represents the variable as more informative than it actually is. Treating missing values equally as a distinct class falsely inflates the value of a variable even more. Considering that the entropy of a data set represents the expected length of a key that would be needed to uniquely identify each distinct item, non-missing values of a variable divide the data into smaller subsets that need smaller keys to index, and the difference in expected key length is the entropy of the variable. Missing values do not sub-divide the data—data with missing values require just as long a key to index with that variable as without it—and thus make the variable as a whole less informative. I created a separate entropy calculation that correctly takes this into account to use for feature selection. This particular calculation is only applicable to nominal data. More work is necessary to adapt the mathematics to numerical data. 

In [5]:
a = [0, 0, 1, 1, 2, 2]
print(
    f'The entropy of a, which has no missing values, is {drv.entropy(a):.3f} bits.')

b = np.array([0, 1, 2, -1, -1, -1])  # pyitlib read -1 as a missing value
print(
    f'The entropy of b, which is missing half its values, is {drv.entropy(b):.3f} bits when calculated by ignoring missing values.'
)
print(
    f'The entropy of b is {drv.entropy(pd.Series(b)):.3f} bits when calculated by treating missing values as a distinct class.'
)
b = np.where(b == -1, np.nan, b)  # code missing values as np.nan
print(
    f'The entropy of b is {ent.entropy(b):.3f} bits when calculated with missing values in mind.'
)

The entropy of a, which has no missing values, is 1.585 bits.
The entropy of b, which is missing half its values, is 1.585 bits when calculated by ignoring missing values.
The entropy of b is 1.792 bits when calculated by treating missing values as a distinct class.
The entropy of b is 0.792 bits when calculated with missing values in mind.


<IPython.core.display.Javascript object>

In [6]:
drv.entropy(df['q2'])

array(0.86813234)

<IPython.core.display.Javascript object>

In [7]:
ent.entropy(df['q2'])

0.315390137859471

<IPython.core.display.Javascript object>

In [8]:
ent.entropy(df).describe()

count    2603.000000
mean        0.249387
std         0.233377
min         0.000255
25%         0.087877
50%         0.182209
75%         0.337961
max         2.922291
dtype: float64

<IPython.core.display.Javascript object>

In [9]:
ent.normalized_entropy(df).describe()

count    2603.000000
mean        0.171020
std         0.139985
min         0.000255
25%         0.062566
50%         0.137574
75%         0.243618
max         0.949562
dtype: float64

<IPython.core.display.Javascript object>

In [10]:
cols = df.columns
ent.info_variation(df[cols[0]], df[cols[1]])

1.3967889524996573

<IPython.core.display.Javascript object>

In [11]:
dist.squareform(pdist(df[cols[:5]].values.T, ent.info_variation))

array([[0.        , 1.39678895, 0.85873146, 0.77348966, 0.80217984],
       [1.39678895, 0.        , 1.36070029, 1.30574991, 1.33855463],
       [0.85873146, 1.36070029, 0.        , 0.73885013, 0.76436172],
       [0.77348966, 1.30574991, 0.73885013, 0.        , 0.48673748],
       [0.80217984, 1.33855463, 0.76436172, 0.48673748, 0.        ]])

<IPython.core.display.Javascript object>

In [12]:
ent.mutual_info(df[cols[1]], df[cols[2]])

0

<IPython.core.display.Javascript object>

Hamming distance is a popular information-theoretic measure of distance between points composed of categorical data. The SciPy implementation treats NaN values almost the same as any other value, with the exception that two NaNs in the same coordinate are treated as not equal. There is a certain logic to this, but it's inconsistent with the spirit of my revised entropy calculation.

In [13]:
# demo limitations of scipy dissimilarities for categorical data
test_matrix = [[0, 0, 1], [0, 1, 0], [0, 1, 2],
               [np.nan, 1, 2], [np.nan, np.nan, 2]]
print('(Normalized) Hamming distance treats NaN the same as any other value')
print(dist.squareform(pdist(test_matrix, dist.hamming)))

(Normalized) Hamming distance treats NaN the same as any other value
[[0.         0.66666667 0.66666667 1.         1.        ]
 [0.66666667 0.         0.33333333 0.66666667 1.        ]
 [0.66666667 0.33333333 0.         0.33333333 0.66666667]
 [1.         0.66666667 0.33333333 0.         0.66666667]
 [1.         1.         0.66666667 0.66666667 0.        ]]


<IPython.core.display.Javascript object>

Let's assume that we're comparing points in a space that follows the idealized assumptions from the original applications for Hamming distance:

- There are no missing values
- All values are binary
- All values are balanced, i.e., $p_0 = p_1 = \frac{1}{2}$
- All values are (statistically) independent

In this setting, Hamming distance has the effect of measuring the size of the smallest set containing both points from the collection of sets determined by filtering on the coordinate values of points: If two points are binary keys indexing a data set, Hamming distance is the (base-2) logarithm of the number of keys with values matching either point (except when the two points are identical). For every value that is equal between two points, the size of the smallest set containing them both shrinks by half (1 bit). Framed this way, SciPy's practice of incrementing (non-normalized) Hamming distance by 1 for every NaN coordinate is reasonable. But I could just as easily frame it the other direction: for every value that is not equal between two points, the size of the smallest set containing both doubles. Then it isn't so clear that SciPy's Hamming distance is doing the right thing.

Alternately, I could simply exclude any variables missing values from the comparison: missing values cut the space under consideration in half rather than doubling the containing set. This could have the side effect of exaggerating the similarity between points with few non-missing values in common. Points with many values in common have more opportunities to disagree, and therefore to be further away just because they aren't missing values. But at least this variation is consistent with both increasing _and_ decreasing the size of the smallest set containing both points—missing values do neither.

In [14]:
a = np.array([0, 0, 0, 1, 1, 1, 0])
b = np.array([0, 0, 1, 0, 1, 1, 1])
hamming_ignore = ent.hamming_variation_func(
    'uniform', 'binary', 'ignore', np.array([a, b])
)
print(hamming_ignore(a, b))

c = np.array([0, 0, 0, np.nan, 1, 1, 0])
d = np.array([0, 0, np.nan, 0, 1, 1, 1])
print(hamming_ignore(c, d))

0.42857142857142855
0.2


<IPython.core.display.Javascript object>

Framing Hamming distance as a measure of the smallest set containing both points inspires a more nuanced measure for when the various assumptions I stated are violated:

- When values match, the smallest set containing both points decreases in proportion to that value's probability, which may not be half. So two points are not be particularly close when they have the same value as most of the population, and two points are closer when they share an unusual value.
- When values don't match for a non-binary nominal variable, the smallest set containing both is the union of the data with those two values, not the whole population. Two profiles with different minority answers are in a smaller common set than they would be if one of them had a more common answer.
- On any variable with an ordinal scale, the smallest common set for two points is the closed interval between them. This provides a basis for extending Hamming distance to handle continuous variables by using differences of quantiles, and a more theory-driven alternative to Gower distance for mixed-type data.

This interpretation also allows for an additional nuance: Correlated variables in combination reduce the size of the smallest common set less than uncorrelated variables. However, it could be expensive to calculate the distance between points while accounting for this. The simplest approach would be to filter the data set for values matching either point and count the rows in the result. Left at that, the result would not be a proper metric, since the smallest set containing a point and itself contains at least one point, and thus the "distance" would not be 0. That might be corrected by checking for equality of all shared values first and calling the distance 0 if all coordinates match. However, it may be preferable not to. If two identical points are part of a large set of other identical points in a population, we don't necessarily want to consider them close.

In [15]:
# Hamming set size calculation prototype (limited to nominal variables)
def get_set_hamming(data):
    if type(data) == pd.core.frame.DataFrame:
        X = data.values.T
    else:
        X = np.asanyarray(data).T
    n_data = X.shape[1]
    n_features = X.shape[0]

    @nb.jit
    def dist_func(a, b):
        match_combined = np.full(n_data, True)
        n_non_nan = 0
        n_match = 0

        for i in range(n_features):
            if not np.isnan(a[i]) and not np.isnan(b[i]):
                nans = np.isnan(X[i])
                match_a = X[i] == a[i]
                match_b = X[i] == b[i]
                match_either = match_a | match_b | nans
                match_combined = match_combined & match_either

        sum_combined = np.sum(match_combined)
        if sum_combined == 0:
            return 1
        else:
            return np.log2(np.sum(match_combined)) / np.log2(n_data)

    return dist_func

<IPython.core.display.Javascript object>

Each of the set-measure-based Hamming distances described above increase distances in the presense of missing values. We sometimes want to minimize the influence of missing values. For this case, I created a family of Hamming distances that can be configured to suit various theoretical and practical goals.

- **weight**: Pattern of weights to apply to each feature
    - *uniform*: All features have equal weight
    - *entropy*: Each feature's weight is proportional to it's (missing-value-aware) entropy in reference data
    - *normalized entropy*: Each feature's weight is proportional to it's entropy in reference data relative to perfect entropy for the number of unique non-missing values in that feature
- **dist**(ance): Calculation of partial distances for non-missing values
    - *binary*: Traditional Hamming distance; 0 for matches, 1 for non-matches
    - *entropy*: Measures the smallest set containing both values within each feature (missing-value-aware) in reference data
- **missing_dist**(ance): Calculation of partial distances for missing values
    - *ignore*: Calculate as if the features with missing values do not exist
    - *max*: Maximum partial distance (1), similar to SciPy implementation
    - *uniform*: Impute what the mean distance for the feature would be if non-missing values were uniformly distributed, i.e., $\frac{k-1}{k}$ for a feature with $k$ unique categories
    - *mean*: Impute what the mean distance actually is between non-missing values in reference data, including distinguishing between cases where both values being compared are missing vs. only one of the two


These functions also require a reference data set for configuration, sometimes to learn value distributions, and always to know how many features to anticipate. They also accept a `missing_ind` parameter to specify different codes for missing values.

I test every combination of these parameters to find a metric that minimizes correlation with the number of missing values in each row.

In [16]:
# test correlation with missing values for various hamming distance configurations
n_pairs = 100000  # approximate
test_sample = df.sample(utl.ifloor(np.sqrt(n_pairs * 2))).values
n_missing_values = pdist(
    test_sample, lambda a, b: np.sum(missing_mask(a) | missing_mask(b))
)

hamming_param_values = {
    'weight': ['uniform', 'entropy', 'normalized entropy'],
    'dist': ['binary', 'entropy'],
    'missing_dist': ['ignore', 'max', 'uniform', 'mean'],
    'data': [test_sample],
    'missing_ind': [MISSING_IND],
}

weight_list = []
dist_list = []
missing_list = []
tau_list = []

for params in ps.param_grid(hamming_param_values):
    hamming_dist = ent.hamming_variation_func(**params)
    dists = pdist(test_sample, hamming_dist)
    tau, p = st.kendalltau(dists, n_missing_values)

    weight_list.append(params['weight'])
    dist_list.append(params['dist'])
    missing_list.append(params['missing_dist'])
    tau_list.append(tau)

missing_correlation_df = pd.DataFrame(
    {
        'weight': weight_list,
        'dist': dist_list,
        'missing_dist': missing_list,
        'tau': tau_list,
        'abs_tau': np.abs(tau_list),
    }
)

<IPython.core.display.Javascript object>

In [17]:
missing_correlation_df.sort_values('abs_tau')

,weight,dist,missing_dist,tau,abs_tau
11,entropy,binary,mean,0.001848,0.001848
19,normalized entropy,binary,mean,0.017754,0.017754
3,uniform,binary,mean,0.041315,0.041315
8,entropy,binary,ignore,-0.168310,0.168310
16,normalized entropy,binary,ignore,-0.215509,0.215509
18,normalized entropy,binary,uniform,0.608756,0.608756
10,entropy,binary,uniform,0.616932,0.616932
0,uniform,binary,ignore,-0.628561,0.628561
2,uniform,binary,uniform,0.724781,0.724781
7,uniform,entropy,mean,0.781830,0.781830


<IPython.core.display.Javascript object>

The configurations with `dist="binary"` consistently correlate least with missing values. `dist="entropy"` metrics include distributions of missing values in the calculation, so it's no surprise that it has a strong relationship with missing values—after all, that's exactly what I created the missing-value-aware entropy for, in the context of feature comparisons.

It's less clear why configurations using `missing_dist="mean"` should be less correlated with missing values than other options—especially `ignore`. By ignoring missing values entirely, the `ignore` metrics proportionally increase the influence of the remaining non-missing values, creating substantial negative correlations. Dropping rows with missing values from a data set can bias an analysis, and dropping columns is no different. The `mean` and `uniform` options both impute partial distances for missing values, and the approach used by `mean` is more context-aware than `uniform`—like the difference betwen interpolation and mean imputation.

`weight="entropy"` metrics correlate less with missing values compared to similar metrics using either `normalized entropy` or `uniform`. `entropy` and `normalized entropy` both down-weight features with more missing values (in addition to features with imbalanced classes), and `normalized entropy` compresses the range of weights compared to `entropy`.

In [19]:
# set measure Hamming distances on full data set & sample of data
hamming_set = get_set_hamming(df)
hamming_set_sample = get_set_hamming(df.sample(5000))

# set measure Hamming distance assuming variables are independently distributed
hamming_set_ind = ent.hamming_variation_func(
    'uniform', 'entropy', 'max', df, MISSING_IND
)

# Hamming distance chosen to correlate least with missing values
hamming_decorr = ent.hamming_variation_func(
    'entropy', 'binary', 'mean', df, MISSING_IND
)

<IPython.core.display.Javascript object>

In [21]:
# sample hamming distances
n_pairs = len(df)
test_sample = df.sample(utl.ifloor(np.sqrt(n_pairs))).values

n_missing_values = pdist(
    test_sample, lambda a, b: np.sum(missing_mask(a) | missing_mask(b))
)
data = {
    'scipy': pdist(test_sample, dist.hamming),
    'set': pdist(test_sample, hamming_set),
    'set_sample': pdist(test_sample, hamming_set_sample),
    'set_ind': pdist(test_sample, hamming_set_ind),
    'decorr': pdist(test_sample, hamming_decorr),
}
hamming_df = pd.DataFrame(data)

<IPython.core.display.Javascript object>

In [22]:
hamming_df.describe()

,scipy,set,set_sample,set_ind,decorr
count,33930.000000,33930.000000,33930.000000,33930.000000,33930.000000
mean,0.937432,0.454855,0.587436,0.894585,0.523454
std,0.067802,0.266864,0.289103,0.126908,0.020664
min,0.522858,0.062262,0.000000,0.110570,0.431033
25%,0.930081,0.062262,0.345705,0.878567,0.512890
50%,0.969650,0.510835,0.575926,0.955804,0.520908
75%,0.978871,0.675265,0.807218,0.971945,0.531956
max,0.998847,0.939799,1.000000,0.998456,0.676982


<IPython.core.display.Javascript object>

In [23]:
hamming_df.corr()

,scipy,set,set_sample,set_ind,decorr
scipy,1.000000,0.776294,-0.442256,0.991072,0.014585
set,0.776294,1.000000,-0.314022,0.774750,0.005069
set_sample,-0.442256,-0.314022,1.000000,-0.452729,0.101978
set_ind,0.991072,0.774750,-0.452729,1.000000,-0.088539
decorr,0.014585,0.005069,0.101978,-0.088539,1.000000


<IPython.core.display.Javascript object>

In [24]:
subsample = df.sample(min(len(df), 1000)).values

<IPython.core.display.Javascript object>

In [26]:
scipy_time = utl.runtimer()
distances = pdist(subsample, dist.hamming)
scipy_time()

runtime: 0:00:06.570011


<IPython.core.display.Javascript object>

In [27]:
sample_time = utl.runtimer()
distances = pdist(subsample, hamming_set_sample)
sample_time()

runtime: 0:02:53.721412


<IPython.core.display.Javascript object>

In [28]:
ind_time = utl.runtimer()
distances = pdist(subsample, hamming_set_ind)
ind_time()

runtime: 0:00:08.696891


<IPython.core.display.Javascript object>

In [29]:
decorr_time = utl.runtimer()
distances = pdist(subsample, hamming_decorr)
decorr_time()

runtime: 0:00:13.451311


<IPython.core.display.Javascript object>

Even when using a small subsample of the reference data, the variation directly using set size is abysmally slow (not to mention the negative relationship of the sample-based set measure distance with it's population counterpart). A suitable data structure might accelerate the computation, such as a materialized OLAP cube.